In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from gensim import corpora,models, similarities
import nltk
import codecs
from nltk.corpus import stopwords

/usr/local/lib/python2.7/dist-packages/gensim-0.13.3-py2.7-linux-x86_64.egg/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [4]:
sentences = []
    
with codecs.open("/home/work/Data/raw.para/judicial.en.split",encoding="utf8") as inputfile:
    for line in inputfile:
         if line != "\n":
                sentences.append(line)

# remove common words and tokenize
stoplist = set(stopwords.words('english'))
texts = [[word for word in sentence.strip().split(" ") if word not in stoplist]
         for sentence in sentences]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]
#remove empty string after filtering put low frequency words
texts = [text for text in texts if len(text)!=0]
print len(texts)
from pprint import pprint  # pretty-printer
#pprint(texts)


808


In [5]:
#using training corpus to build dictionary
dictionary = corpora.Dictionary(texts)
# store the dictionary, for future reference
dictionary.save('/home/work/test.dict') 
print(dictionary)

Dictionary(849 unique tokens: [u'Arbitration', u'limited', u'Maizidian', u'caused', u'(5)']...)


In [6]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/home/work/test.mm', corpus)
#print(corpus)

In [7]:
#creating tfidf transformation and initialise the model
tfidf = models.TfidfModel(corpus) 
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf]

In [8]:
index = similarities.SparseMatrixSimilarity(corpus_tfidf, num_features=12)

In [9]:
doc = "2) \"ZHANG Fan\" invested RMB 10,000,000 to Beijing Zhongjin Xinsheng Investment Center (limited partnership)"
vec_bow = dictionary.doc2bow(doc.lower().split())
bow_tfidf = tfidf[vec_bow]

#for doc in corpus_tfidf:
#    print(doc)
bow_lsi = lsi[bow_tfidf]
#print(vec_lsi)
#sims = index[tfidf[vec_bow]]
#print(list(enumerate(sims)))

In [13]:
index = similarities.MatrixSimilarity(corpus_lsi)

In [14]:
sims = index[bow_lsi]
res = sorted(enumerate(sims), key=lambda item: -item[1])
print len(res)
for r in res[:30]:
    print texts[r[0]],r[1]

808
[u'It', u'Financial', u'Statements', u'office', u'Accounting', u'Measures', u'Beijing', u'Law', u'Offices', u'formulated', u'Beijing', u'Justice', u'Bureau', u'Beijing', u'Finance', u'Bureau,', u'relevant', u'provisions', u'annual', u'inspection', u'lawyers,', u'financial', u'office', u'dated', u'2013', u'operation', u'2013', u'material'] 1.0
[u'LI', u'Dan', u'(signature)', u'(hereinafter', u'referred'] 0.999999
[u'(8)', u'Vehicle', u'maintenance', u'expenses'] 0.999994
[u'We', u'financial', u'statements', u'Beijing', u'Zhong', u'Xin', u'Law', u'Office', u'(hereinafter', u'referred', u'including', u'Balance', u'Sheet', u'dated', u'December', u'31,', u'2013,', u'Profit', u'&', u'Profit', u'Distribution', u'Statement', u'2013,', u'well', u'Statement', u'Basic', u'Figures', u'Annual'] 0.999963
[u'(4)', u'public', u'welfare'] 0.999953
[u'24'] 0.999913
[u'24'] 0.999913
[u'Short-Term'] 0.999859
[u'2.', u'Profit', u'&', u'Profit', u'Distribution', u'Statement'] 0.999848
[u'Including:(1)',